In [ ]:
#
# Universidad EAFIT 
# 2025-1
# SI7003 - NLP - Lecture 07
#

In [ ]:
# instalar dependencias
#%pip install openai langchain langchain_community streamlit
#%pip install faiss-cpu
# instalar ChromaDB como retriever 
#%pip install chromadb
#%pip install --index-url https://pypi.org/simple  openai langchain langchain_community streamlit faiss-cpu chromadb tiktoken

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached tiktoken-0.9.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.7 kB)
Using cached tiktoken-0.9.0-cp312-cp312-macosx_11_0_arm64.whl (1.0 MB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
# paso 1:
import os
from langchain.chat_models import ChatOpenAI

# Configurar la API Key (puedes establecerla como variable de entorno)
#os.environ["OPENAI_API_KEY"] = "your-openai-api-key"
# bash: export OPENAI_API_KEY="your-openai-api-key"

# Inicializar el modelo GPT-4
llm = ChatOpenAI(model_name="gpt-4")


/var/folders/0z/3s22b27525jc3bm0m923h36h3wg489/T/ipykernel_24424/964916048.py:10: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-4")


In [5]:
# Inicializar la Base de Datos Vectorial

import chromadb

# Inicializar la base de datos ChromaDB
chroma_client = chromadb.PersistentClient(path="./chroma_db")

# Crear una colección en ChromaDB
collection = chroma_client.get_or_create_collection(name="chatbot_knowledge")

#


In [6]:
# Agregar Datos de Referencia a la Base de Datos

documents = [
    {"id": "1", "text": "La inteligencia artificial es el campo de la informática que estudia cómo crear sistemas capaces de realizar tareas que requieren inteligencia humana."},
    {"id": "2", "text": "GPT-4 es un modelo de lenguaje de OpenAI basado en la arquitectura de transformers."},
    {"id": "3", "text": "LangChain es una biblioteca para desarrollar aplicaciones basadas en modelos de lenguaje."}
]

# Agregar documentos a ChromaDB
for doc in documents:
    collection.add(ids=[doc["id"]], documents=[doc["text"]])


/Users/andrestrepo/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:05<00:00, 14.5MiB/s]


In [7]:
# Crear un Retriever con ChromaDB

from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings

# Inicializar embeddings con OpenAI
embeddings = OpenAIEmbeddings()

# Crear el VectorStore con ChromaDB
vectorstore = Chroma(persist_directory="./chroma_db", embedding_function=embeddings)

# Crear el retriever para buscar documentos relevantes
retriever = vectorstore.as_retriever()


/var/folders/0z/3s22b27525jc3bm0m923h36h3wg489/T/ipykernel_24424/789104924.py:7: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()
/var/folders/0z/3s22b27525jc3bm0m923h36h3wg489/T/ipykernel_24424/789104924.py:10: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(persist_directory="./chroma_db", embedding_function=embeddings)


In [8]:
# paso 2

from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

# Crear memoria de la conversación
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Integrar memoria en la cadena conversacional

#chat_chain = ConversationalRetrievalChain.from_llm(llm, memory=memory)

# fix con un retriever en chromadb
chat_chain = ConversationalRetrievalChain.from_llm(llm, memory=memory,retriever=retriever)


/var/folders/0z/3s22b27525jc3bm0m923h36h3wg489/T/ipykernel_24424/2690817752.py:7: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [9]:
# Paso 3: Construir la Función del Chatbot

def chat_with_gpt(user_input):
    response = chat_chain({"question": user_input})
    return response["answer"]


In [13]:
# Ejecución del Chatbot
#%pip install streamlit
#%pip install tiktoken

!streamlit run app.py


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://10.161.40.245:8501

  For better performance, install the Watchdog module:

  $ xcode-select --install
  $ pip install watchdog
            
/Users/andrestrepo/Documents/repos_personal/si7003nlp_eafit/.venv/lib/python3.12/site-packages/langchain/chat_models/__init__.py:33: LangChainDeprecationWarning: Importing chat models from langchain is deprecated. Importing from langchain will no longer be supported as of langchain==0.2.0. Please import from langchain-community instead:

`from langchain_community.chat_models import ChatOpenAI`.

To install langchain-community run `pip install -U langchain-community`.
  warnings.warn(
/Users/andrestrepo/Documents/repos_personal/si7003nlp_eafit/.venv/lib/python3.12/site-packages/langchain/chat_models/__init__.py:33: LangChainDeprecationWarning: Importing chat models from langchain is deprecated. Importing from langchain will no longer be

In [ ]:
# mejoras propuestas

# Agregar RAG (Retrieval-Augmented Generation) con bases de datos vectoriales y tus propios datos.
# Integrar API externas (ejemplo: Wikipedia, bases de datos de conocimiento).
# Incluir Voice-to-Text para hacer un chatbot por voz.
